In [1]:
import pandas as pd
import os
import re

# Define file paths
ptype_file_path = r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\3. Mar'25 Final period type.xlsx"
tagging_file_path = r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\ADJN_Only_Cases_Amt_df.csv"

try:
    # Load ptype_df (Excel)
    ptype_df = pd.read_excel(ptype_file_path, sheet_name='p type sheet - 58447 cases', engine="openpyxl")
    
    
    # Load tagging_data_working (CSV)
    tagging_data_working = pd.read_csv(tagging_file_path)
    
    # Clean column names in tagging_data_working
    tagging_data_working.columns = tagging_data_working.columns.str.strip()
    
    # Ensure 'Approved Performa' exists in both dataframes
    if 'Approved Performa' not in ptype_df.columns or 'Approved Performa' not in tagging_data_working.columns:
        raise KeyError("❌ 'Approved Performa' column missing in one of the files.")

    # Step 1: Update 'star/ leader/ IVE/TS' column based on exact word match
    def classify_service(service_name):
        if pd.isna(service_name):  # Handle missing values
            return ""  # ✅ Fixed indentation here

        name_lower = str(service_name).strip().lower()

        # Exact matches using regex
        patterns = {
            'Leader': r'^\s*leader\s*$',  
            'Star': r'^\s*star\s*$',  
            'TS': r'^\s*trustseal\s+(1\s*year|2\s*years|3\s*years)\s*$'  # Strictly match only these patterns
        }

        for category, pattern in patterns.items():
            if re.fullmatch(pattern, name_lower):
                return category

        # **Special condition for "Exporter" (check if it exists anywhere in the string)**
        if re.search(r'\bexporter\b', name_lower):  
            return "IVE"

        return ""

    ptype_df['star/ leader/ IVE/TS'] = ptype_df['Service Name'].apply(classify_service)

    # Step 2: Create new column 'net dv'
    ptype_df['net dv'] = (ptype_df['Deal Amount'] - ptype_df['Other Receipt Amount']) + ptype_df['Total NFR Reversal W/O Tax']

    # Step 3: Merge with tagging_data_working to get 'r/nr as per tagging data working' & 'Total Amount'
    ptype_df = ptype_df.merge(
        tagging_data_working[['Approved Performa', 'Relv/Not Relv', 'Total Amount']],
        on='Approved Performa',
        how='left'
    ).rename(columns={'Relv/Not Relv': 'r/nr as per tagging data working'})

    # Remove duplicate column issue (if any)
    ptype_df = ptype_df.loc[:, ~ptype_df.columns.duplicated()]

    # Step 4: Create 'net cn/adjn amt' (only for 'R' cases)
    ptype_df['net cn/adjn amt'] = ptype_df.apply(
        lambda row: row['Total Amount'] if row['r/nr as per tagging data working'] == 'R' else None,
        axis=1
    )

    # Step 5: Drop 'Total Amount' column
    ptype_df.drop(columns=['Total Amount'], inplace=True)

    # Convert 'net cn/adjn amt' to numeric, setting errors='coerce' to turn invalid values into NaN
    ptype_df['net cn/adjn amt'] = pd.to_numeric(ptype_df['net cn/adjn amt'], errors='coerce')

    # Now, subtract directly (Pandas will handle NaN properly)
    ptype_df['net dv - net cn/adjn (only for r cases)'] = ptype_df['net dv'] - ptype_df['net cn/adjn amt']

    # Save updated file
    output_file = ptype_file_path.replace(".xlsx", "_Updated.xlsx")
    ptype_df.to_excel(output_file, index=False, engine="openpyxl")

    # Print Summary
    print("✅ File processed successfully! Updated file saved as:\n", output_file)
    print("\n📊 Summary:")
    print("Count of 'net dv':", ptype_df['net dv'].count())
    print("Sum of 'net dv':", ptype_df['net dv'].sum())
    print("Count of 'net cn/adjn amt' (non-null):", ptype_df['net cn/adjn amt'].notna().sum())
    print("Sum of 'net cn/adjn amt' (excluding NaN):", ptype_df['net cn/adjn amt'].sum())

except FileNotFoundError as e:
    print("❌ Error: File not found. Check the file path.")
except KeyError as e:
    print(f"❌ Key Error: {e}")
except Exception as e:
    print(f"❌ An error occurred: {e}")

✅ File processed successfully! Updated file saved as:
 D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\3. Mar'25 Final period type_Updated.xlsx

📊 Summary:
Count of 'net dv': 58447
Sum of 'net dv': 2119272546.6300004
Count of 'net cn/adjn amt' (non-null): 1124
Sum of 'net cn/adjn amt' (excluding NaN): 36274636.66254237


In [2]:
print(ptype_df.head())

   Renewal Entry No.  Approved Performa    GL User  Service ID  \
0            7079641            6477119   12173902         340   
1            7297109            6623707   83251287         340   
2            9141324            8829962    6962854         302   
3            9149629            8844586  124977345         362   
4            9230613            8810818     293433         280   

        Service Name star/ leader/ IVE/TS Service Start Date Service End Date  \
0  TrustSEAL 3 years                   TS         2025-06-21       2025-10-18   
1  TrustSEAL 3 years                   TS         2025-07-30       2025-09-19   
2    LS City Listing                              2024-12-10       2027-10-20   
3       IM LEADER 10                              2024-12-13       2025-04-25   
4          Maximiser                              2025-01-17       2028-11-26   

  Period Type  Deal Amount  ...  current start dt-previous end dt  \
0          PB      6702.54  ...                

In [3]:
# Ensure 'net dv - net cn/adjn (only for r cases)' is numeric
ptype_df['net dv - net cn/adjn (only for r cases)'] = pd.to_numeric(
    ptype_df['net dv - net cn/adjn (only for r cases)'], errors='coerce'
)

# Create 'final ext/conv as per dv and net value' using vectorized conditions
ptype_df['final ext/conv as per dv and net value'] = 'non ext/conv'
ptype_df.loc[
    ptype_df['net dv - net cn/adjn (only for r cases)'].between(-10, 10, inclusive='both'),
    'final ext/conv as per dv and net value'
] = 'ext/conv'

ptype_df.to_csv(r'D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\final ext conv as per dv and net value.csv', index=False)
print('final ext conv as per dv and net value saved')

final ext conv as per dv and net value saved


In [4]:
#Filter rows where 'final ext/conv as per dv and net value' is 'ext/conv'
filtered_df = ptype_df[ptype_df['final ext/conv as per dv and net value'] == 'ext/conv']
filtered_df.count()

filtered_df['Renewal Entry No.'].to_csv(r'D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\Ptype_Renewal Entry No.csv', index=False)
print('Renewal Entry No.csv Saved')

Renewal Entry No.csv Saved


In [5]:
# filtered_df for concatenation

filtered_df = filtered_df.copy()  # Ensure it's a separate DataFrame

#Concatination of GLUser & Service ID
filtered_df.loc[:, 'GL|Serv ID'] = filtered_df['GL User'].astype(str) + "|" + filtered_df['Service ID'].astype(str)

#Save concate_glusr_serviceID
filtered_df.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\concate_glusr_serviceID.csv",index=False)
print('concate_glusr_serviceID saved')

# Extract unique 'GL User' values
unique_gl_users = filtered_df[['GL User']].drop_duplicates()

# Define output file path
output_csv_path = r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\Unique ext_conv GL Users.csv"

# Save to CSV (without index for clean output)
unique_gl_users.to_csv(output_csv_path, index=False)

print(f"✅ File saved successfully: {output_csv_path}")


concate_glusr_serviceID saved
✅ File saved successfully: D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\Unique ext_conv GL Users.csv


In [6]:
import pandas as pd

# Define file path
dl_file_path = r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\DeferredList_05.04.25 02.42 PM.xlsb"

# Load the file
DL_df = pd.read_excel(dl_file_path, sheet_name='DeferredList_05.04.25 02.42 PM', engine="pyxlsb")

# Automatically detect and convert all date columns
for col in DL_df.columns:
    if "date" in col.lower():  # Identify columns with 'date' in the name
        DL_df[col] = pd.to_datetime(DL_df[col], origin='1899-12-30', unit='D', errors='coerce')

# Update 'Remarks_New' where 'Actual Active' contains 'Ankit' (case insensitive)
DL_df['Remarks_New'] = ""
DL_df.loc[DL_df['Actual Active'].astype(str).str.contains('ankit', case=False, na=False), 'Remarks_New'] = "Ankit"

# Sample manual MMM-YY filter date
manual_mmm_yy = "Apr-25"

# Convert to datetime (1st day of the month for comparison)
manual_date = pd.to_datetime(manual_mmm_yy, format="%b-%y") + pd.offsets.MonthBegin(0)

# Ensure 'Service Start Date' is in datetime format and normalize
if 'Service Start Date' in DL_df.columns:
    DL_df['Service Start Date'] = pd.to_datetime(DL_df['Service Start Date'], errors='coerce')

    # Update 'Remarks_New' to "Future" where Service Start Date > manual_date
    DL_df.loc[DL_df['Service Start Date'] >= manual_date, 'Remarks_New'] = "Future"

print("✅ 'Remarks_New' column updated successfully!")

✅ 'Remarks_New' column updated successfully!


In [7]:
DL_df.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\remarks_new_updated.csv", index=False)
print("remarks_new_updated saved")

remarks_new_updated saved


In [8]:
import pandas as pd

# Filter cases where 'Remarks_New' is blank (not 'Ankit' or 'Future')
DL_df_blank_remarks = DL_df[~DL_df['Remarks_New'].isin(['Ankit', 'Future'])].copy()

# Print count of filtered cases
print(f"Total cases with blank Remarks_New: {len(DL_df_blank_remarks)}")

# Print DL_df_blank_remarks.columns
print(DL_df_blank_remarks.columns)

# Save the filtered data to CSV
DL_df_blank_remarks.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\Remarks_New_Blank.csv", index=False)

print("✅ 'Remarks_New_Blank.csv' saved successfully!")

Total cases with blank Remarks_New: 11163
Index(['Renewal Entry No.', 'Clubbed Collecction Calc', 'Cust to Serv ID',
       'Service ID', 'Service Name', 'GL User', 'Type', 'Remarks',
       'Service Period', 'Period Type', 'To-From Renewal', 'Advice Date',
       'Start Date', 'Service Start Date', 'Service End Date',
       'Renewal End Date', 'Renewal Entered Date', 'WO Completed on',
       'Stop Date', 'Download  Date', 'Revenue Recognition', 'Employee ID',
       'Approved Performa', 'Work Order', 'Dept ID', 'Sub Dept ID',
       'Vertical ID', 'Location ID', 'Cost Centre', 'No of Periods',
       'Revenue Account', 'Deal Amount', 'Other Receipt Amount',
       'Total NFR Reversal W/O Tax', 'PB Amount', 'OT Amount',
       'Amount to Defer', 'Total Amount', 'Amount Transferred',
       'Direct Revenue', 'Revenue Reversed', 'Remaining Amount',
       'Collection Amount', 'Clubbed Collection', 'Old Collection Amount',
       'Last Remaining Amount', 'Outstanding Amount', 'Active',


In [9]:
# remove unnecessary space before or after column names
DL_df_blank_remarks.columns = DL_df_blank_remarks.columns.str.strip()
filtered_df.columns = filtered_df.columns.str.strip()

#Make sure 'Renewal Entry No.' is into str format before lookup operation
DL_df_blank_remarks['Renewal Entry No.'] = DL_df_blank_remarks['Renewal Entry No.'].astype(str)
filtered_df['Renewal Entry No.'] = filtered_df['Renewal Entry No.'].astype(str)

#print after dtype conversion
print(DL_df_blank_remarks.info())
print(filtered_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 11163 entries, 0 to 12278
Data columns (total 56 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Renewal Entry No.             11163 non-null  object        
 1   Clubbed Collecction Calc      11163 non-null  float64       
 2   Cust to Serv ID               11163 non-null  int64         
 3   Service ID                    11163 non-null  int64         
 4   Service Name                  11163 non-null  object        
 5   GL User                       11163 non-null  int64         
 6   Type                          11163 non-null  object        
 7   Remarks                       0 non-null      object        
 8   Service Period                11163 non-null  object        
 9   Period Type                   11163 non-null  object        
 10  To-From Renewal               11163 non-null  int64         
 11  Advice Date                   110

In [10]:
# Perform the lookup by merging on 'Renewal Entry No.'
DL_df_blank_remarks = DL_df_blank_remarks.merge(
    filtered_df[['Renewal Entry No.']],  # Only matching entries
    on='Renewal Entry No.', 
    how='left',
    indicator=True  # Adds a column to show match status
)

# Create the new column: If found in filtered_df, keep 'Renewal Entry No.', else keep it blank
DL_df_blank_remarks['REN entry vlookup'] = DL_df_blank_remarks.apply(
    lambda row: row['Renewal Entry No.'] if row['_merge'] == 'both' else '', axis=1
)

# Drop the merge indicator column
DL_df_blank_remarks.drop(columns=['_merge'], inplace=True)

# Check results
print(DL_df_blank_remarks[['Renewal Entry No.', 'REN entry vlookup']].head())

DL_df_blank_remarks.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\REN entry vlookup.csv", index=False)
print('REN entry vlookup updated')

  Renewal Entry No. REN entry vlookup
0            679747                  
1            963106                  
2           1667050                  
3           1973125                  
4           2328439                  
REN entry vlookup updated


In [11]:
# filtered_df for concatenation
REN_entry_vlookup = DL_df_blank_remarks.copy()  # Ensure it's a separate DataFrame

#Concatination of GLUser & Service ID
filtered_df.loc[:, 'GL|Serv ID'] = filtered_df['GL User'].astype(str) + "|" + filtered_df['Service ID'].astype(str)

#Save concate_glusr_serviceID
filtered_df.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\concate_glusr_serviceID.csv",index=False)
print('concate_glusr_serviceID saved')

concate_glusr_serviceID saved


In [12]:
import pandas as pd

# Step 1: Filter rows where 'REN entry vlookup' is blank
REN_entry_vlookup = DL_df_blank_remarks[DL_df_blank_remarks['REN entry vlookup'] == ""].copy()

# Step 2: Add a new column 'GL|Serv ID' by combining 'GL User' and 'Service ID'
REN_entry_vlookup['GL|Serv ID'] = REN_entry_vlookup['GL User'].astype(str) + "|" + REN_entry_vlookup['Service ID'].astype(str)

# Check the first few rows of the new DataFrame
print(REN_entry_vlookup.head())

# GL|Serv ID pipe creation
REN_entry_vlookup.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\pipe_created.csv", index=False)
print('GL|Serv ID pipe created')

  Renewal Entry No.  Clubbed Collecction Calc  Cust to Serv ID  Service ID  \
0            679747                       0.0           183055         280   
1            963106                       0.0           396351         266   
2           1667050                       0.0           396351         266   
3           1973125                       0.0           183055         280   
4           2328439                       0.0           396351         266   

       Service Name  GL User           Type Remarks Service Period  \
0         Maximiser    55438        Renewal     NaN         ThreeY   
1  Leading Supplier    55438        Renewal     NaN         Annual   
2  Leading Supplier    55438        Renewal     NaN         Annual   
3         Maximiser    55438        Renewal     NaN         ThreeY   
4  Leading Supplier    55438  Period Upsell     NaN         ThreeY   

  Period Type  ...  Actual Active Remarks.1 Opening  \
0          PB  ...            NaN       Yes      No   


In [13]:
import pandas as pd

# Step 1: Read the saved file
file_path = r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\concate_glusr_serviceID.csv"
console_df = pd.read_csv(file_path)

# Step 2: Merge on 'GL|Serv ID' to perform lookup and add an indicator
REN_entry_vlookup = REN_entry_vlookup.merge(
    console_df[['GL|Serv ID']],  
    on='GL|Serv ID', 
    how='left',
    indicator=True  # Adds a merge indicator column
)

# Step 3: Create the new column 'lookup of comb from console'
REN_entry_vlookup['lookup of comb from console'] = REN_entry_vlookup.apply(
    lambda row: row['GL|Serv ID'] if row['_merge'] == 'both' else "", axis=1
)

# Step 4: **Remove duplicates based on all columns**
REN_entry_vlookup_unique = REN_entry_vlookup.drop_duplicates()

# Step 5: Count matched and blank cases
matched_count = (REN_entry_vlookup_unique['lookup of comb from console'] != "").sum()
blank_count = (REN_entry_vlookup_unique['lookup of comb from console'] == "").sum()

# Print the counts
print(f"Matched Cases (Unique): {matched_count}")
print(f"Blank Cases (Unique): {blank_count}")

# Drop the indicator column as it's no longer needed
REN_entry_vlookup_unique = REN_entry_vlookup_unique.drop(columns=['_merge'])

# Check the results
print(REN_entry_vlookup_unique.head())

# Save REN_entry_vlookup_unique
REN_entry_vlookup_unique.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\REN_entry_vlookup_unique.csv", index=False)
print('REN_entry_vlookup_unique saved')

Matched Cases (Unique): 2483
Blank Cases (Unique): 8342
  Renewal Entry No.  Clubbed Collecction Calc  Cust to Serv ID  Service ID  \
0            679747                       0.0           183055         280   
1            963106                       0.0           396351         266   
2           1667050                       0.0           396351         266   
3           1973125                       0.0           183055         280   
4           2328439                       0.0           396351         266   

       Service Name  GL User           Type Remarks Service Period  \
0         Maximiser    55438        Renewal     NaN         ThreeY   
1  Leading Supplier    55438        Renewal     NaN         Annual   
2  Leading Supplier    55438        Renewal     NaN         Annual   
3         Maximiser    55438        Renewal     NaN         ThreeY   
4  Leading Supplier    55438  Period Upsell     NaN         ThreeY   

  Period Type  ...  Remarks.1 Opening Exclude Entry in

In [14]:
# Step 6: Filter rows where 'lookup of comb from console' has a value (non-blank)
filtered_lookup = REN_entry_vlookup_unique[REN_entry_vlookup_unique['lookup of comb from console'] != ""].copy()

# Step 7: Convert 'Renewal Entry No.' to numeric (if it's not already)
filtered_lookup['Renewal Entry No.'] = pd.to_numeric(filtered_lookup['Renewal Entry No.'], errors='coerce')

# Step 8: Sort by 'Renewal Entry No.' in descending order
filtered_lookup = filtered_lookup.sort_values(by='Renewal Entry No.', ascending=False)

# Check the first few rows after filtering and sorting
print(filtered_lookup.head())

filtered_lookup.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\filtered_lookup_descending.csv", index=False)
print('filtered_lookup_descending Saved')

       Renewal Entry No.  Clubbed Collecction Calc  Cust to Serv ID  \
11076            9399739                       0.0          2894726   
7622             9398753                       0.0          2894180   
10499            9398733                       0.0          2894162   
3974             9398721                       0.0          2894154   
4502             9397906                       0.0          2893689   

       Service ID            Service Name    GL User            Type Remarks  \
11076         350  Verified Exporter-Gold  215027829            Sale     NaN   
7622          228    Mini Dynamic Catalog   63684103            Sale     NaN   
10499         340       TrustSEAL 3 years  157199468  Product Upsell     NaN   
3974          318       TrustSEAL 2 years   13050471            Sale     NaN   
4502          364        Preferred Leader   17245704  Product Upsell     NaN   

      Service Period Period Type  ...  Remarks.1 Opening  \
11076         Annual          PB

In [15]:
console_df.head()

,Renewal Entry No.,Approved Performa,GL User,Service ID,Service Name,star/ leader/ IVE/TS,Service Start Date,Service End Date,Period Type,Deal Amount,...,EXT/CONVERSION,final duration to be used,FINAL STRT DATE.,FINAL END DT.,FINAL END DY.,FINAL PERIOD TYPE,net dv,r/nr as per tagging data working,net cn/adjn amt,GL|Serv ID
0,7079641,6477119,12173902,340,TrustSEAL 3 years,TS,2025-06-21,2025-10-18,PB,6702.54,...,Extension (period 1 day),previous duration,2022-06-22,2025-06-20,1095,ThreeY,6702.54,R,6702.54,12173902|340
1,7297109,6623707,83251287,340,TrustSEAL 3 years,TS,2025-07-30,2025-09-19,PB,4116.10,...,Extension (period 1 day),previous duration,2022-07-31,2025-07-29,1095,ThreeY,4116.10,R,4116.10,83251287|340
2,9149629,8844586,124977345,362,IM LEADER 10,NaN,2024-12-13,2025-04-25,PB,36849.16,...,conversion,current duration,2024-12-13,2025-04-25,134,Annual,36849.16,R,36849.15,124977345|362
3,9230613,8810818,293433,280,Maximiser,NaN,2025-01-17,2028-11-26,PB,197901.69,...,non ext/conv,current duration,2025-01-17,2028-11-26,1410,ThreeY,192958.47,R,192958.48,293433|280
4,9238778,8937736,158827661,364,Preferred Leader,NaN,2026-01-18,2026-01-18,PB,610.17,...,Extension (period 1 day),previous duration,2025-01-18,2026-01-17,365,Annual,610.17,R,610.17,158827661|364


In [16]:
# Step 1: Create a copy of console_df
console_df = console_df.copy()

# Step 2: Ensure necessary columns in both DataFrames are in datetime format
console_df['Service Start Date'] = pd.to_datetime(console_df['Service Start Date'])
filtered_lookup['Service Start Date'] = pd.to_datetime(filtered_lookup['Service Start Date'])
filtered_lookup['Service End Date'] = pd.to_datetime(filtered_lookup['Service End Date'])

# Step 3: Sort filtered_lookup by 'Renewal Entry No.' in descending order
filtered_lookup = filtered_lookup.sort_values(by='Renewal Entry No.', ascending=False)

# Step 4: Drop duplicates based on 'GL|Serv ID' (keeping the first occurrence)
filtered_lookup = filtered_lookup.drop_duplicates(subset=['GL|Serv ID'], keep='first')

# Step 5: Merge 'filtered_lookup' with 'console_df' on 'GL|Serv ID'
console_df = console_df.merge(
    filtered_lookup[['GL|Serv ID', 'Service Start Date', 'Service End Date']], 
    on='GL|Serv ID', 
    how='left',
    suffixes=('', '_prev')  # Prevents _x and _y suffix confusion
)

# Step 6: Assign values from lookup to newly created columns
console_df['prev renewal start dt'] = console_df['Service Start Date_prev']
console_df['prev renewal end dt'] = console_df['Service End Date_prev']

# Step 7: Drop unnecessary columns created during merge
console_df.drop(columns=['Service Start Date_prev', 'Service End Date_prev'], inplace=True)

# Step 8: Verify the DataFrame structure
print(console_df[['GL|Serv ID', 'Service Start Date', 'prev renewal start dt', 'prev renewal end dt']].head())

# console_df top rows
console_df.head()

#Save prev renewal_dates_updated
console_df.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\prev renewal_dates_updated.csv", index=False)
print('prev renewal_dates_updated')

      GL|Serv ID Service Start Date prev renewal start dt prev renewal end dt
0   12173902|340         2025-06-21            2022-06-22          2025-06-20
1   83251287|340         2025-07-30            2022-07-31          2025-07-29
2  124977345|362         2024-12-13            2024-10-28          2025-10-25
3     293433|280         2025-01-17                   NaT                 NaT
4  158827661|364         2026-01-18            2025-01-18          2026-01-17
prev renewal_dates_updated


In [17]:
#Ensure both columns are in datetime format before subtraction
console_df['Service Start Date'] = pd.to_datetime(console_df['Service Start Date'])
console_df['prev renewal end dt'] = pd.to_datetime(console_df['prev renewal end dt'])

# Compute the difference and store in a new column
console_df['current start dt-previous end dt'] = (console_df['Service Start Date'] - console_df['prev renewal end dt']).dt.days

# Display the updated DataFrame
print(console_df[['Service Start Date', 'prev renewal end dt', 'current start dt-previous end dt']].head())

# Save current start dt-previous end dt updated
console_df.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\current start dt-previous end dt.csv", index=False)
print('current start dt-previous end dt updated')

  Service Start Date prev renewal end dt  current start dt-previous end dt
0         2025-06-21          2025-06-20                               1.0
1         2025-07-30          2025-07-29                               1.0
2         2024-12-13          2025-10-25                            -316.0
3         2025-01-17                 NaT                               NaN
4         2026-01-18          2026-01-17                               1.0
current start dt-previous end dt updated


In [18]:
# Step 1: Ensure 'current start dt-previous end dt' column exists and is numeric
console_df['current start dt-previous end dt'] = pd.to_numeric(console_df['current start dt-previous end dt'], errors='coerce')

# Step 2: Apply conditions to determine 'EXT/CONVERSION' values
console_df['EXT/CONVERSION'] = console_df['current start dt-previous end dt'].apply(
    lambda x: 'Extension (period 1 day)' if x == 1 else ('Non-Extension' if pd.isna(x) else 'Conversion')
)

# Step 3: Display results
print(console_df[['current start dt-previous end dt', 'EXT/CONVERSION']].head())

console_df.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\EXT_CONVERSION_updated.csv", index=False)
print('EXT_CONVERSION or Non-Conversion updated')

   current start dt-previous end dt            EXT/CONVERSION
0                               1.0  Extension (period 1 day)
1                               1.0  Extension (period 1 day)
2                            -316.0                Conversion
3                               NaN             Non-Extension
4                               1.0  Extension (period 1 day)
EXT_CONVERSION or Non-Conversion updated


In [19]:
# Step 1: Apply conditions to determine 'final duration to be used' values
console_df['final duration to be used'] = console_df['EXT/CONVERSION'].apply(
    lambda x: 'current duration' if x == 'Conversion' else 
              ('previous duration' if x == 'Extension (period 1 day)' else 
               'current duration')
)

# Step 2: Display results
print(console_df[['EXT/CONVERSION', 'final duration to be used']].head())

console_df.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\final duration to be used.csv", index=False)
print('final duration to be used updated')

             EXT/CONVERSION final duration to be used
0  Extension (period 1 day)         previous duration
1  Extension (period 1 day)         previous duration
2                Conversion          current duration
3             Non-Extension          current duration
4  Extension (period 1 day)         previous duration
final duration to be used updated


In [20]:
# Ensure the columns are in datetime format
date_columns = ['Service Start Date', 'Service End Date', 'prev renewal start dt', 'prev renewal end dt']
for col in date_columns:
    console_df[col] = pd.to_datetime(console_df[col], errors='coerce')  # Handle any invalid dates

# Apply conditional logic based on 'final duration to be used'
console_df['FINAL STRT DATE'] = console_df.apply(
    lambda row: row['Service Start Date'] if row['final duration to be used'] == 'current duration' 
    else row['prev renewal start dt'], axis=1
)

console_df['FINAL END DT'] = console_df.apply(
    lambda row: row['Service End Date'] if row['final duration to be used'] == 'current duration' 
    else row['prev renewal end dt'], axis=1
)

# Calculate 'FINAL DURATION' as difference + 1
console_df['FINAL DURATION'] = (console_df['FINAL END DT'] - console_df['FINAL STRT DATE']).dt.days + 1

# Display the updated DataFrame with new columns
print(console_df[['final duration to be used', 'FINAL STRT DATE', 'FINAL END DT', 'FINAL DURATION']].head())

console_df.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\FINAL DURATION.csv",index=False)
print('FINAL DURATION.csv saved')

  final duration to be used FINAL STRT DATE FINAL END DT  FINAL DURATION
0         previous duration      2022-06-22   2025-06-20            1095
1         previous duration      2022-07-31   2025-07-29            1095
2          current duration      2024-12-13   2025-04-25             134
3          current duration      2025-01-17   2028-11-26            1410
4         previous duration      2025-01-18   2026-01-17             365
FINAL DURATION.csv saved


In [21]:
# Ensure 'FINAL PERIOD TYPE' column exists
if 'FINAL PERIOD TYPE' not in console_df.columns:
    console_df['FINAL PERIOD TYPE'] = ''

# Define function to update 'FINAL PERIOD TYPE'
def update_final_period_type(row):
    # Case 1: If 'Period Type' is 'OT', set 'FINAL PERIOD TYPE' to 'Monthly'
    if row['Period Type'] == 'OT':
        return 'Monthly'

    # Case 2: If 'star/ leader/ IVE/TS' is 'TS', extract duration from 'Service Name'
    if row['star/ leader/ IVE/TS'] == 'TS':
        if '1' in row['Service Name']:
            return 'Annual'
        elif '2' in row['Service Name']:
            return 'TwoY'
        elif '3' in row['Service Name']:
            return 'ThreeY'

    # Case 3: If 'Service Name' is 'Maximiser' and 'Deal Amount' is in [82,500, 90,000]
    if row['Service Name'] == 'Maximiser' and row['Deal Amount'] in [82500, 90000]:
        return 'TwoY'

    # Case 4: If 'star/ leader/ IVE/TS' is in ['IVE', 'Leader', 'Star']
    if row['star/ leader/ IVE/TS'] in ['IVE', 'Leader', 'Star']:
        if row['EXT/CONVERSION'] == 'Non-Extension' and 32 <= row['FINAL DURATION'] <= 200:
            return 'Bi-Annual'
        elif row['EXT/CONVERSION'] == 'Conversion' and 32 <= row['FINAL DURATION'] <= 200:
            return 'Annual'

    # Case 5: Final update based on 'FINAL DURATION'
    final_period_type = row.get('FINAL PERIOD TYPE', '')  # Use .get() to avoid KeyError
    if pd.isna(final_period_type) or final_period_type == '':
        if row['FINAL DURATION'] <= 31:
            return 'Monthly'
        elif 32 <= row['FINAL DURATION'] <= 450:
            return 'Annual'
        elif row['FINAL DURATION'] > 450:
            return 'ThreeY'
    
    return final_period_type  # Keep existing value if conditions don't match

# Apply function to update 'FINAL PERIOD TYPE'
console_df['FINAL PERIOD TYPE'] = console_df.apply(update_final_period_type, axis=1)

# Display updated DataFrame
print(console_df[['Service Name', 'Deal Amount', 'FINAL DURATION', 'FINAL PERIOD TYPE']].head())

final_period_type_counts = console_df.groupby('FINAL PERIOD TYPE').size().reset_index(name='Count')
print(final_period_type_counts)

        Service Name  Deal Amount  FINAL DURATION FINAL PERIOD TYPE
0  TrustSEAL 3 years      6702.54            1095            ThreeY
1  TrustSEAL 3 years      4116.10            1095            ThreeY
2       IM LEADER 10     36849.16             134            Annual
3          Maximiser    197901.69            1410            ThreeY
4   Preferred Leader       610.17             365            Annual
  FINAL PERIOD TYPE  Count
0            Annual    495
1           Monthly     60
2            ThreeY    336
3              TwoY    207


In [22]:
console_df.to_csv(r"D:\Bhuwan Data\Bhuwan\Desktop\Adhoc data of BPnA team\Sonali\Grouping2\Final period type_Updated_Final.csv",index=False)
print("Final period type_Updated_Final Saved")

Final period type_Updated_Final Saved
